## CSE 4705: Assignment 02 - Arad to Bucharest - BFS, DFS, UCS, GBFS, A*


## Problem 1
[100] Write a routine that solves the problem of finds a travel path of cities from from Arad to Bucharest in Romania, as discussed in class. Do this using each of the following approaches (points shown in brackets):

1. [15] Breadth First Search (BFS)
2. [10] Depth First Search (DFS)
3. [25] Uniform Cost Search (UCS)
4. [25] Greedy Best First Search (GBFS)
5. [25] A*


You will use the map from Lecture 03 - Informed Search which shows the major cities in Romania and the distances between them for those cities that are directly connected.  Also, you will use the straight-line-distances shown in the adjacent table for your heuristic function, $h(n)$ for GBFS and A*.  A screenshot of the relevant slide is given below.  Data structures that store this information, romania_map and sld_to_bucharest, have been provided so you can access and apply this data in your algorithm implementations.  Details of these data structures are given below.

<img src="images/romania_cities.png" width="800" height="600">

### Output for Each Routine

Each of your routines should return an output or set of outputs that clearly indicates the following:

1. The sequence of cities from Arad to Bucharest.  (Make sure the cities, Arad and Bucharest are explicitly listed as the first and last cities in your output.)  One suggestion is to return this output in the form of a list.
2. Cost to travel to each city from its predecessor.  
3. Total cost for the path.  

In the case of A* and Uniform Cost Search, your routines should return the *cheapest path*.  However, that will not necessarily be the case for BFS, DFS, or GBFS.  (Why not?)

### Romania Graph

You will use the data structure stored in the romania_map, assigned below to implement the search across the various cities to find a path from Arad to Bucharest.

Some details about romania_map:
- A dictionary of dictionaries
- The outer dictionary is as follows: each key is a city and the value for that city is a nested dictionary of cities to which the said city is directly connected.
- The nested dictionary contains the cities to which the parent key is directly connected (keys) and the corresponding distances from the parent city to those respective cities (values).
- For example, for the city Oradea, we have a key in the outer dictionary (Oradea), and the associated value is a dictionary containing the Zerind and Sibiu as keys, where for each of these the values are the distances from Oradea to these respective cities.



In [608]:
romania_map = {
    'Oradea':{'Zerind':71, 'Sibiu':151},
    'Zerind':{'Oradea':71, 'Arad':75},
    'Arad':{'Zerind':75, 'Sibiu':140, 'Timisoara':118},
    'Timisoara':{'Arad':118, 'Lugoj':111},
    'Lugoj':{'Timisoara':111, 'Mehadia':70},
    'Mehadia':{'Lugoj':70, 'Dobreta':75},
    'Dobreta':{'Mehadia':75, 'Craiova':120},
    'Sibiu':{'Oradea':151, 'Fagaras':99, 'Rimnicu Vilcea':80, 'Arad':140},
    'Rimnicu Vilcea':{'Sibiu':80, 'Pitesti':97, 'Craiova':146},
    'Craiova':{'Rimnicu Vilcea':146, 'Pitesti':138, 'Dobreta':120},
    'Fagaras':{'Sibiu':99, 'Bucharest':211},
    'Pitesti':{'Rimnicu Vilcea':97, 'Bucharest':101, 'Craiova':138},
    'Neamt':{'Iasi':87},
    'Giurgiu':{'Bucharest':90},
    'Bucharest':{'Pitesti':101, 'Fagaras':211, 'Urziceni':85, 'Giurgiu':90},
    'Iasi':{'Neamt':87, 'Vaslui':92},
    'Urziceni':{'Bucharest':85, 'Vaslui':142, 'Hirsova':98},
    'Vaslui':{'Iasi':92, 'Urziceni':142},
    'Hirsova':{'Urziceni':98, 'Eforie':86},
    'Eforie':{'Hirsova':86}
}

In [609]:
print(romania_map)

{'Oradea': {'Zerind': 71, 'Sibiu': 151}, 'Zerind': {'Oradea': 71, 'Arad': 75}, 'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118}, 'Timisoara': {'Arad': 118, 'Lugoj': 111}, 'Lugoj': {'Timisoara': 111, 'Mehadia': 70}, 'Mehadia': {'Lugoj': 70, 'Dobreta': 75}, 'Dobreta': {'Mehadia': 75, 'Craiova': 120}, 'Sibiu': {'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80, 'Arad': 140}, 'Rimnicu Vilcea': {'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146}, 'Craiova': {'Rimnicu Vilcea': 146, 'Pitesti': 138, 'Dobreta': 120}, 'Fagaras': {'Sibiu': 99, 'Bucharest': 211}, 'Pitesti': {'Rimnicu Vilcea': 97, 'Bucharest': 101, 'Craiova': 138}, 'Neamt': {'Iasi': 87}, 'Giurgiu': {'Bucharest': 90}, 'Bucharest': {'Pitesti': 101, 'Fagaras': 211, 'Urziceni': 85, 'Giurgiu': 90}, 'Iasi': {'Neamt': 87, 'Vaslui': 92}, 'Urziceni': {'Bucharest': 85, 'Vaslui': 142, 'Hirsova': 98}, 'Vaslui': {'Iasi': 92, 'Urziceni': 142}, 'Hirsova': {'Urziceni': 98, 'Eforie': 86}, 'Eforie': {'Hirsova': 86}}


### Heuristic Function Data - Straight-Line Distances to Bucharest

You will use the dictionary below as your resource for retrieving straight-line distance data for implementing the GBFS and A* algorithms.

In [610]:
print(romania_map['Oradea'])

{'Zerind': 71, 'Sibiu': 151}


In [611]:
sld_to_Bucharest = {'Arad':366,
                    'Bucharest':0,
                    'Craiova':160,
                    'Dobreta':242,
                    'Eforie':161,
                    'Fagaras':176,
                    'Giurgiu':77,
                    'Hirsova':151,
                    'Iasi':226,
                    'Lugoj':244,
                    'Mehadia':241,
                    'Neamt':234,
                    'Oradea':380,
                    'Pitesti':100,
                    'Rimnicu Vilcea':193,
                    'Sibiu':253,
                    'Timisoara':329,
                    'Urziceni':80,
                    'Vaslui':199,
                    'Zerind':374
                   }

#Data Structures Setup

Graph and Nodes

In [612]:
class Node:
    def __init__(self, name):
        self.name = name
        self.distance = float('inf')
        self.predecessor = None

    def __lt__(self, other):
        return self.distance < other.distance

    def __repr__(self):
        return self.name

class Graph:
    def __init__(self, V, E = ()):
        self.V = V
        self.E = E


    def neighbors(self, node):
        return self.E.get(node, [])

    def __len__(self):
        return len(self.V)

In [613]:
map_verts = {}
for v in romania_map.keys():
  map_verts[v] = Node(v)

map_edges = {map_verts[city]: [(map_verts[neighbor], cost) for neighbor, cost in connections.items()] for city, connections in romania_map.items()}
romania_graph = Graph(map_verts, map_edges)

Priority Queue

In [614]:
class PriorityQueue(object):
    def __init__(self):
        self.queue = []

    def __str__(self):
        return ' '.join([str(i) for i in self.queue])

    def isEmpty(self):
        return len(self.queue) == 0

    def insert(self, data):
        self.queue.append(data)

    def count(self, input):
        count = 0
        for i in self.queue:
          if i[1].name == input[1].name:
            count += 1
        return count

    def remove_larger_duplicates(self, input):
      for i in self.queue:
        if i[1].name == input[1].name and i[0] < input[0]:
          self.queue.remove(i)
          break

    def get(self):
       item = min(self.queue)
       self.queue.remove(item)

       return item

### 1. BFS Implementation

Provide your implementation of the BFS Search below.

In [615]:
def path_to_goal(start_node, end_node):
  s = end_node
  out = []
  while s != start_node:
    out.append(s.name)
    s = s.predecessor
  out.append(start_node.name)
  return out[::-1]

In [616]:
def BFS(graph, start_node):
  visited_nodes = set()
  frontier = []

  start_node.predecessor = None
  visited_nodes.add(start_node)
  frontier.append(start_node)

  while frontier:
    current_node = frontier.pop(0)
    for neighbor_node, distance in graph.neighbors(current_node):
      if neighbor_node not in visited_nodes:
        neighbor_node.predecessor = current_node
        if neighbor_node.name == 'Bucharest':
          return path_to_goal(start_node, neighbor_node)
        visited_nodes.add(neighbor_node)
        frontier.append(neighbor_node)
  return None

In [617]:
def get_dist(path):
  dist = 0
  for i in range(len(path)-1):
    dist += romania_map[path[i]][path[i+1]]
  return dist

In [618]:
path_bfs = BFS(romania_graph, romania_graph.V['Arad'])
print(path_bfs)
print(get_dist(path_bfs))

['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
450


### 2.  DFS Implementation

Provide your implementation of the DFS Search below.

In [619]:
def DFS(graph, start_node):
  visited_nodes = set()
  frontier = []

  start_node.predecessor = None
  visited_nodes.add(start_node)
  frontier.append(start_node)

  while frontier:
    current_node = frontier.pop()
    for neighbor_node, distance in graph.neighbors(current_node):
      if neighbor_node not in visited_nodes:
        neighbor_node.predecessor = current_node
        if neighbor_node.name == 'Bucharest':
          return path_to_goal(start_node, neighbor_node)
        visited_nodes.add(neighbor_node)
        frontier.append(neighbor_node)
  return None

In [620]:
path_dfs = DFS(romania_graph, romania_graph.V['Arad'])
print(path_dfs)
print(get_dist(path_dfs))

['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Dobreta', 'Craiova', 'Pitesti', 'Bucharest']
733


### 3. UCS Implementation

Provide your implementation of the UCS Search below.

In [627]:
def UCS(graph, start_node):
  start_node.predecessor = None
  frontier = PriorityQueue()
  frontier.insert((0, start_node))

  explored_nodes = []
  while not frontier.isEmpty():
    priority, current_node = frontier.get()
    explored_nodes.append(current_node)
    if current_node.name == 'Bucharest':
      return path_to_goal(start_node, current_node)
    for neighbor_node, distance in graph.neighbors(current_node):
      if neighbor_node not in explored_nodes:
        neighbor_node.predecessor = current_node
        f_n = priority + distance
        if frontier.count((f_n, neighbor_node)) > 0:
          frontier.remove_larger_duplicates((f_n, neighbor_node))
        else:
          frontier.insert((f_n, neighbor_node))

  return None

In [628]:
path_ucs = UCS(romania_graph, romania_graph.V['Arad'])
print(path_ucs)
print(get_dist(path_ucs))

['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
418


### 4. GBFS Implementation

Provide your implementation of the GBFS Search below.

In [629]:
def GBFS(graph, start_node):
  start_node.predecessor = None
  frontier = PriorityQueue()
  frontier.insert((0, start_node))

  explored_nodes = []

  while not frontier.isEmpty():
    priority, current_node = frontier.get()
    explored_nodes.append(current_node)
    if current_node.name == 'Bucharest':
      return path_to_goal(start_node, current_node)
    for neighbor_node, distance in graph.neighbors(current_node):
      if neighbor_node not in explored_nodes:
        neighbor_node.predecessor = current_node
        f_n = sld_to_Bucharest[neighbor_node.name]
        frontier.insert((f_n, neighbor_node))
        frontier.remove_larger_duplicates((f_n, neighbor_node))
  return None

In [630]:
path_gbfs = GBFS(romania_graph, romania_graph.V['Arad'])
print(path_gbfs)
print(get_dist(path_gbfs))

['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
450


### 5. A* Implementation

Provide your implementation of the A* Algorithm below.

In [631]:
def AStar(graph, start_node):
  start_node.predecessor = None
  frontier = PriorityQueue()
  frontier.insert((0, start_node))

  explored_nodes = []

  while not frontier.isEmpty():
    priority, current_node = frontier.get()
    explored_nodes.append(current_node)
    if current_node.name == 'Bucharest':
      return path_to_goal(start_node, current_node)
    for neighbor_node, distance in graph.neighbors(current_node):
      if neighbor_node not in explored_nodes:
        neighbor_node.predecessor = current_node
        f_n = sld_to_Bucharest[neighbor_node.name] + distance
        frontier.insert((f_n, neighbor_node))
        frontier.remove_larger_duplicates((f_n, neighbor_node))
  return None

In [632]:
path_astar = AStar(romania_graph, romania_graph.V['Arad'])
print(path_astar)
print(get_dist(path_astar))

['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
418
